In [1]:
from ase.io import read, write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import PeriodicSite

from tqdm import tqdm
from pymatgen.core import Composition, Structure
from typing import List, Tuple

import re, joblib, json
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from pyxtal.symmetry import Group
import pickle
import pandas as pd
from ase.io import read, write
from ase import Atoms
from ase.data import chemical_symbols
from ase.db import connect
import numpy as np
import sys, os
from spglib import get_spacegroup, find_primitive, standardize_cell
import pymatgen 
from pymatgen.io.ase import AseAtomsAdaptor
import ast
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import roc_curve, auc,accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # for cross-validation
from datetime import datetime 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler
from pymatgen.core import Composition, Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifParser
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pymatgen.io.cif import CifWriter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import math
from tqdm import tqdm
from sklearn.neighbors import KernelDensity

sys.path.append('/home/energy/mahpe/Published_code/Dis-CSP/dis_csp')

from dis_csp.model import CrystalDataset, VAE
from dis_csp.generation import decode_samples, generate_wyckoffgene, get_cif_lines, generate_cif_files
from dis_csp.csp_filter import structure_validity,oxidation_state_validity


2025-07-16 07:39:26.329487: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 07:39:26.528641: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-16 07:39:26.565611: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 07:39:26.565665: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 07:39:26.565697: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
latent_space_path = '/home/energy/mahpe/Published_code/Dis-CSP/Decoded_data/z_samples_ran.npy'
model_dir = 'New_Kl5_ICSD_dis_site_middle_KL_element1000_lr_5e-06_epochs_2500_batch_64_test_0.2_val_0.1'
best_model = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load the VAE model ###
if best_model:
    vae_dict = torch.load(model_dir+'/best_vae_model.pth',map_location=torch.device('cpu'))
else: # use exit model
    vae_dict = torch.load(model_dir+'/exit_vae_model.pth',map_location=torch.device('cpu'))

feature_dim = 183
wyckoff_dim = 9
crystal_dim = 236
kernel = [5,3,2]
stride = [2,3,2]
max_filter = 16
latent_dim = 256
vae_eval = VAE(feature_dim, wyckoff_dim, crystal_dim,verbose=False,kernel=kernel,stride=stride)
vae_eval.load_state_dict(vae_dict['model'])

# Decode the latent space samples
#latent_space_path = 'Decoded_data/z_samples_kde_optimized.npy'
cif_save_path = 'Decoded_data/Generated_cif'
print(f"Loading latent space samples from {latent_space_path}")
z_samples = np.load(latent_space_path)

if 'Y_scaler' in vae_dict:
    scaler_Y = vae_dict['Y_scaler']
else:
    # Load the scaler if not present in the model dictionary
    print('Loading Y_scaler from file Y_scaler.gz')
    scaler_Y = joblib.load('Y_scaler.gz')

### Load the latent space samples ###
z_samples = np.load(latent_space_path)

# Get the decoded samples
save_dict = decode_samples(z_samples,vae_eval,scaler_Y,device='cpu')

Loading latent space samples from /home/energy/mahpe/Published_code/Dis-CSP/Decoded_data/z_samples_ran.npy
Loading Y_scaler from file Y_scaler.gz


/home/energy/mahpe/Published_code/Dis-CSP/dis_csp/generation.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'disordered_site': F.sigmoid(torch.tensor(disordered_site)).cpu().detach().numpy(),


In [3]:
# Generate Wyckoff genes
mixiter = None
element_acc = 0.05  
disorder_acc = 0.5
all_wyckoffgenes,sma_int, failed_int = generate_wyckoffgene(save_dict,
                             max_iter=mixiter,
                             element_acc=element_acc,
                             disorder_acc=disorder_acc,
                             shift_frac_coord=True,
                             verbose=False)
if mixiter is not None:
    print(f"Symmetry Matching Accuracy: {100-sma_int/mixiter*100:.2f}%")
    print(f"Number of failed items: {failed_int} of {mixiter} processed, Procentage: {failed_int/mixiter*100:.2f}%")
else:
    print(f"Symmetry Matching Accuracy: {100- sma_int/len(save_dict['spacegroup'])*100:.2f}%")
    print(f"Number of failed items: {failed_int} of {len(save_dict['spacegroup'])} processed, Procentage: {failed_int/len(save_dict['spacegroup'])*100:.2f}%")


100%|██████████| 27008/27008 [00:32<00:00, 824.49it/s] 

Symmetry Matching Accuracy: 8.39%
Number of failed items: 715 of 27008 processed, Procentage: 2.65%


In [7]:
print(f"Number of failed items: {failed_int} of {len(save_dict['spacegroup'])} processed, Procentage: {100-failed_int/len(save_dict['spacegroup'])*100:.2f}%")

Number of failed items: 715 of 27008 processed, Procentage: 97.35%


In [8]:
len_all_structures = len(save_dict['spacegroup'])


all_cifs,cifs__wyckoffgene = generate_cif_files(all_wyckoffgenes,maxiter=len_all_structures,validity_primitive=False,symmetry_analyzer=False, verbose=False, two_oxidation_state=False)

Processing 27008 structures out of 1552


  0%|          | 0/1552 [00:00<?, ?it/s]/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'orthorhombic' is not interpreted for space group Spacegroup(62, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(141, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(140, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'cubic' is not interpreted for space group Spacegroup(224, setting=1). This may result in wrong settin

Failed to parse 0 structures out of 27008 Procentage: 100.00%
Validity check failed for 293 structures out of 27008 Procentage: 98.92%
Oxidation state check failed for 425 structures out of 27008 Procentage: 98.43%
Total valid structures: 968 out of 27008 Procentage: 3.58%
